In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
from lib.Plot import plot
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# import pandas as pd
# test = pd.read_csv("drive/MyDrive/datasets/allAnnotations.csv")
# test

import numpy as np
X_train = np.load("drive/MyDrive/datasets/X_train.npy")
X_val = np.load("drive/MyDrive/datasets/X_val.npy")
y_train_e = np.load("drive/MyDrive/datasets/y_train_e.npy")
y_val_e = np.load("drive/MyDrive/datasets/y_val_e.npy" )

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Mar 30 21:23:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    34W / 250W |  15981MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from keras.layers import Input, Conv2D, Lambda, MaxPool2D, UpSampling2D, AveragePooling2D, ZeroPadding2D
from keras.layers import Activation, Flatten, Dense, Add, Multiply, BatchNormalization, Dropout
from keras.models import Model

In [ ]:
class RAN():
    def __init__(self, input_shape, n_classes, activation, p=1, t=2, r=1):
        self.input_shape = input_shape
        self.n_classes = n_classes
        self.activation = activation
        self.p = p
        self.t = t
        self.r = r
        
    def build_model(self):
        input_data = Input(shape = self.input_shape)

        conv_1 = Conv2D(64, (7,7), strides=2, activation=self.activation)(input_data)

        max_pool = MaxPool2D((2,2), strides=2)(conv_1)

        res_unit_1 = self.residual_unit(max_pool, [32, 32, 64])
        attention_model_1 = self.attention_model(res_unit_1, [32, 32, 64])

        res_unit_2 = self.residual_unit(attention_model_1, [64, 64, 128])
        attention_model_2 = self.attention_model(res_unit_2, [64, 64, 128])

        res_unit_3 = self.residual_unit(attention_model_2, [128, 128, 256])
        attention_model_3 = self.attention_model(res_unit_3, [128, 128, 256])

        res_unit_4 = self.residual_unit(attention_model_3, [256, 256, 512])
        res_unit_4 = self.residual_unit(res_unit_4, [256, 256, 512])
        res_unit_4 = self.residual_unit(res_unit_4, [256, 256, 512])
        #res_unit_4 = self.residual_unit(res_unit_4, [32, 32, 64])

        avg_pool = AveragePooling2D((7,7))(res_unit_4)

        flatten = Flatten()(avg_pool)

        fc_1 = Dense(256, activation = 'relu')(flatten)
        fc_1 = Dropout(0.5)(fc_1)
        fc_2 = Dense(256, activation = 'relu')(fc_1)
        fc_2 = Dropout(0.5)(fc_2)
        fc_3 = Dense(256, activation = 'relu')(fc_2)
        fc_3 = Dropout(0.5)(fc_3)
        fc_4 = Dense(self.n_classes, activation=self.activation)(fc_3)

        model = Model(inputs = input_data, outputs = fc_4)

        return model

    def residual_unit(self, residual_input_data, filters):
        filter1, filter2, filter3 = filters

        batch_norm_1 = BatchNormalization()(residual_input_data)
        activation_1 = Activation('relu')(batch_norm_1)
        conv_1 = Conv2D(filter1, (1,1), strides=1, padding='same')(activation_1)

        batch_norm_2 = BatchNormalization()(conv_1)
        activation_2 = Activation('relu')(batch_norm_2)
        conv_2 = Conv2D(filter2, (3,3), strides=1, padding='same')(activation_2)

        batch_norm_3 = BatchNormalization()(conv_2)
        activation_3 = Activation('relu')(batch_norm_3)
        conv_3 = Conv2D(filter3, (1,1), strides=1, padding='same')(activation_3)

        if residual_input_data.shape[-1] != conv_3.shape[-1]:
            filter_n = conv_3.shape[-1]

            residual_input_data = Conv2D(filter_n, (1,1), strides=1, padding='same')(residual_input_data)

        output = Add()([residual_input_data, conv_3])

        return output

    def attention_model(self, attention_input_data, filters):
        p_input_data = attention_input_data
        for _ in range(self.p):
            p_input_data = self.residual_unit(p_input_data, filters = filters)

        trunk_branch = self.trunk(p_input_data, filters = filters)

        mask_branch = self.mask(p_input_data, filters)

        ar_learning = self.attention_residual_learning(mask_branch, trunk_branch)

        p_data = ar_learning
        for _ in range(self.p):
            p_data = self.residual_unit(p_data, filters)

        return p_data

    def trunk(self, trunk_input_data, filters):
        for _ in range(self.t):
            trunk_input_data = self.residual_unit(trunk_input_data, filters)
        return trunk_input_data

    def mask(self, mask_input_data, filters, m = 3):
        # r = num of residual units between adj pool layers
        # m = num of max pool / linear interpolations to do

        downsample = MaxPool2D(2, strides=2, padding='same')(mask_input_data)
        downsample = self.residual_unit(downsample, filters)

        downsample = MaxPool2D(2, strides=2, padding='same')(downsample)
#         downsample = self.residual_unit(downsample, filters)

#         downsample = MaxPool2D(2, strides=2, padding='same')(downsample)

        middle = downsample
        for _ in range(2*self.r):
            middle = self.residual_unit(middle, filters)

        upsample = UpSampling2D(2)(middle)


        upsample = self.residual_unit(upsample, filters)

        upsample = UpSampling2D(2)(upsample)
#         upsample = self.residual_unit(upsample, filters)

#         upsample = UpSampling2D(2)(upsample)



        conv_filter = upsample.shape[-1]

        conv1 = Conv2D(conv_filter, (1,1), strides=1, padding='same')(upsample)
        conv2 = Conv2D(conv_filter, (1,1), strides=1, padding='same')(conv1)

        sig = Activation('sigmoid')(conv2)

        return sig

    def attention_residual_learning(self, mask_input, trunk_input):
        Mx = Lambda(lambda x: 1 + x)(mask_input) # 1 + mask
        return Multiply()([Mx, trunk_input]) # M(x) * T(x)

In [ ]:
model = RAN(X_train.shape[1:], 47, activation= 'softmax').build_model()
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.optimizers.SGD(learning_rate=0.01),
              metrics=['accuracy'])
#callback = tf.keras.callbacks.LearningRateScheduler(scheduler)


In [ ]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    try:
        from keras import backend as K
    except:
        from tensorflow.keras import backend as K

    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(p) for p in model.non_trainable_weights])

    number_size = 4.0
    if K.floatx() == 'float16':
        number_size = 2.0
    if K.floatx() == 'float64':
        number_size = 8.0

    total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes

In [ ]:
get_model_memory_usage(45, model)

16.729

In [ ]:
callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.01, patience=5, verbose=1,
    mode='max', min_delta=0.0001, cooldown=2, min_lr=0)
epochs = 20
history = model.fit(X_train, y_train_e, batch_size=32, epochs=epochs, validation_data=(X_val, y_val_e), callbacks = [callback])

Epoch 1/20
129/129 [==============================] - 72s 500ms/step - loss: 0.8404 - accuracy: 0.7608 - val_loss: 0.9071 - val_accuracy: 0.7673 - lr: 0.0100
Epoch 2/20
129/129 [==============================] - 61s 469ms/step - loss: 0.8197 - accuracy: 0.7685 - val_loss: 0.8389 - val_accuracy: 0.7819 - lr: 0.0100
Epoch 3/20
129/129 [==============================] - 61s 469ms/step - loss: 0.8084 - accuracy: 0.7741 - val_loss: 0.8806 - val_accuracy: 0.7775 - lr: 0.0100
Epoch 4/20
129/129 [==============================] - 61s 470ms/step - loss: 0.7926 - accuracy: 0.7756 - val_loss: 0.8642 - val_accuracy: 0.7724 - lr: 0.0100
Epoch 5/20
129/129 [==============================] - 61s 470ms/step - loss: 0.8129 - accuracy: 0.7705 - val_loss: 0.8635 - val_accuracy: 0.7812 - lr: 0.0100
Epoch 6/20
129/129 [==============================] - 61s 469ms/step - loss: 0.7855 - accuracy: 0.7773 - val_loss: 0.8685 - val_accuracy: 0.7848 - lr: 0.0100
Epoch 7/20
129/129 [==============================] 

In [ ]:
model.save("drive/MyDrive/datasets/RAN1")
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'drive/MyDrive/datasets/history80.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

INFO:tensorflow:Assets written to: drive/MyDrive/datasets/RAN1/assets


In [ ]:
model = tf.keras.models.load_model("drive/MyDrive/datasets/RAN1")

In [ ]:
callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.1, patience=5, verbose=1,
    mode='max', min_delta=0.0001, cooldown=2, min_lr=0)
epochs = 20
history = model.fit(X_train, y_train_e, batch_size=32, epochs=epochs, validation_data=(X_val, y_val_e), callbacks = [callback])

Epoch 1/20
 94/129 [====================>.........] - ETA: 10s - loss: 1.9222 - accuracy: 0.4548

KeyboardInterrupt: 